In [4]:
import os
import pandas as pd
import SimpleITK as sitk
import numpy as np
import pydicom
import nibabel as nib
from tqdm import tqdm
import matplotlib.pyplot as plt
import shutil

# 1. 전체 프로젝트 경로
BASE_DIR = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection'
# 2. 실제 데이터가 들어있는 old_train 경로
DATA_DIR = os.path.join(BASE_DIR, 'old_train')

# --- 입력 경로 ---
TRAIN_CSV = os.path.join(DATA_DIR, 'train.csv')
BBOX_CSV = os.path.join(DATA_DIR, 'train_bounding_boxes.csv')
IMAGE_DIR = os.path.join(DATA_DIR, 'train_images')    # old_train 폴더 안의 이미지
SEG_DIR = os.path.join(DATA_DIR, 'segmentations')     # old_train 폴더 안의 마스크

# --- 출력/처리 경로 ---
# 변환된 NIfTI 이미지를 저장할 곳 (old_train/converted_images)
CONV_DIR = os.path.join(DATA_DIR, 'converted_images') 
PROC_DIR = os.path.join(DATA_DIR, 'processed_masks')
SUMMARY_DIR = os.path.join(DATA_DIR, 'patient_summaries') 
GOLDEN_DATA_DIR = os.path.join(BASE_DIR, 'golden_dataset')
REPORT_DIR = os.path.join(DATA_DIR, 'alignment_reports') # 결과 저장 폴더

# 필터링된 결과물
FILTERED_TRAIN_CSV = os.path.join(GOLDEN_DATA_DIR, 'filtered_train.csv')
FILTERED_BBOX_CSV = os.path.join(GOLDEN_DATA_DIR, 'filtered_train_bounding_boxes.csv')

# 데이터프레임 로드 (필요시)
# train_df = pd.read_csv(TRAIN_CSV)
# df_bbox = pd.read_csv(BBOX_CSV)

print(f"설정된 이미지 경로: {IMAGE_DIR}")
print(f"설정된 변환 저장 경로: {CONV_DIR}")


설정된 이미지 경로: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\train_images
설정된 변환 저장 경로: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\converted_images


In [5]:
def create_tiered_dataset_v4():
    # A. 환자 그룹 분류 로직
    seg_files = [f for f in os.listdir(SEG_DIR) if f.endswith('.nii') or f.endswith('.nii.gz')]
    seg_ids = set([f.split('.nii')[0] for f in seg_files])
    
    bbox_df = pd.read_csv(BBOX_CSV)
    bbox_ids = set(bbox_df['StudyInstanceUID'].unique())
    
    groups = {
        '1_mask_and_box': seg_ids.intersection(bbox_ids),
        '2_only_mask': seg_ids - bbox_ids,
        '3_only_box': bbox_ids - seg_ids
    }

    # B. 데이터 복사 및 정리
    for g_name, p_ids in groups.items():
        if not p_ids: continue
        print(f"\n📂 {g_name} 그룹 분류 중... (대상: {len(p_ids)}명)")
        
        for pid in tqdm(p_ids):
            # 1. 원본 DICOM 이미지 (train_images)
            src_img = os.path.join(IMAGE_DIR, pid)
            if os.path.exists(src_img):
                target = os.path.join(GOLDEN_DATA_DIR, g_name, 'train_images', pid)
                shutil.copytree(src_img, target, dirs_exist_ok=True)

            # 2. NIfTI 마스크 (segmentations)
            if pid in seg_ids:
                for ext in ['.nii.gz', '.nii']:
                    src_seg = os.path.join(SEG_DIR, pid + ext)
                    if os.path.exists(src_seg):
                        target_dir = os.path.join(GOLDEN_DATA_DIR, g_name, 'segmentations')
                        os.makedirs(target_dir, exist_ok=True)
                        shutil.copy2(src_seg, target_dir)

            # 3. 슬라이스된 .npy 마스크 (processed_masks)
            src_proc = os.path.join(PROC_DIR, pid)
            if os.path.exists(src_proc):
                target = os.path.join(GOLDEN_DATA_DIR, g_name, 'processed_masks', pid)
                shutil.copytree(src_proc, target, dirs_exist_ok=True)

            # 4. 환자별 요약 CSV (patient_summaries)
            if os.path.exists(SUMMARY_DIR):
                summary_files = [f for f in os.listdir(SUMMARY_DIR) if f.startswith(pid)]
                for sf in summary_files:
                    target_dir = os.path.join(GOLDEN_DATA_DIR, g_name, 'patient_summaries')
                    os.makedirs(target_dir, exist_ok=True)
                    shutil.copy2(os.path.join(SUMMARY_DIR, sf), target_dir)

            # 5. 변환된 전체 NIfTI 이미지 (converted_images)
            # 파일 형태(pid_image.nii.gz)일 수도 있고 폴더(pid/)일 수도 있음
            # 먼저 파일 형태를 체크
            conv_file_name = f"{pid}_image.nii.gz"
            src_conv_file = os.path.join(CONV_DIR, conv_file_name)
            src_conv_dir = os.path.join(CONV_DIR, pid)

            if os.path.exists(src_conv_file): # 파일인 경우
                target_dir = os.path.join(GOLDEN_DATA_DIR, g_name, 'converted_images')
                os.makedirs(target_dir, exist_ok=True)
                shutil.copy2(src_conv_file, target_dir)
            elif os.path.exists(src_conv_dir): # 폴더인 경우
                target = os.path.join(GOLDEN_DATA_DIR, g_name, 'converted_images', pid)
                shutil.copytree(src_conv_dir, target, dirs_exist_ok=True)

    print(f"\n✨ 모든 그룹 데이터 정리가 완료되었습니다! 경로: {GOLDEN_DATA_DIR}")

# 실행
create_tiered_dataset_v4()


📂 1_mask_and_box 그룹 분류 중... (대상: 40명)


100%|██████████| 40/40 [00:15<00:00,  2.61it/s]



📂 2_only_mask 그룹 분류 중... (대상: 47명)


100%|██████████| 47/47 [00:17<00:00,  2.75it/s]



📂 3_only_box 그룹 분류 중... (대상: 195명)


100%|██████████| 195/195 [01:00<00:00,  3.22it/s]


✨ 모든 그룹 데이터 정리가 완료되었습니다! 경로: C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset


In [7]:
import os
import torch
import numpy as np
import pydicom
import pandas as pd
import SimpleITK as sitk
from scipy.ndimage import zoom
from tqdm import tqdm
import segmentation_models_pytorch as smp
import torch.nn.functional as F

# --- [1. 경로 및 설정] ---
BASE_DIR = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection'
GOLDEN_DIR = os.path.join(BASE_DIR, 'golden_dataset')
OLD_DIR = os.path.join(BASE_DIR, 'old_train')

WEIGHTS_DIR = os.path.join(GOLDEN_DIR, 'checkpoints') # fold1~5 pth 파일 위치
MASK_SAVE_ROOT = os.path.join(GOLDEN_DIR, 'seg_pseudo_masks') # 최종 결과 폴더

# 🎯 2019명 원본 폴더 및 235명 명단 CSV 경로 설정
DICOM_ROOT_DIR = os.path.join(OLD_DIR, 'train_images')
BBOX_CSV_PATH = os.path.join(OLD_DIR, 'train_bounding_boxes.csv')

TARGET_Z, TARGET_SIZE = 128, 224
WL, WW = 500, 2000
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

os.makedirs(MASK_SAVE_ROOT, exist_ok=True)

# --- [2. 모델 5개 로드] ---
models = []
for f in range(1, 6):
    model_path = os.path.join(WEIGHTS_DIR, f'best_model_fold{f}.pth')
    
    # 가중치 파일이 없으면 에러 메시지 출력 후 종료 방지
    if not os.path.exists(model_path):
        print(f"❌ 모델 가중치가 없습니다: {model_path}")
        continue
        
    m = smp.Unet(encoder_name="resnet34", in_channels=2, classes=8).to(DEVICE)
    m.load_state_dict(torch.load(model_path, map_location=DEVICE))
    m.eval()
    models.append(m)
    
if len(models) == 5:
    print(f"✅ 5-Fold 모델 앙상블 준비 완료")
else:
    print(f"⚠️ {len(models)}개의 모델만 로드되었습니다. 확인이 필요합니다.")

# --- [3. 유틸리티 함수] ---
def apply_windowing(img, wl, ww):
    img_min, img_max = wl - ww // 2, wl + ww // 2
    img = np.clip(img, img_min, img_max)
    return ((img - img_min) / ww).astype(np.float32)

# --- [4. 메인 앙상블 추론 루프] ---
@torch.no_grad()
def run_ensemble_factory():
    if not os.path.exists(BBOX_CSV_PATH):
        print(f"❌ 라벨 파일이 없습니다: {BBOX_CSV_PATH}")
        return
        
    # 🎯 CSV에서 235명 명단 추출
    bbox_df = pd.read_csv(BBOX_CSV_PATH)
    uids = sorted(list(bbox_df['StudyInstanceUID'].astype(str).unique()))
    print(f"\n📖 CSV 로드 완료! 타겟 환자 수: {len(uids)}명")
    print(f"🚀 [old_train/train_images] 폴더에서 앙상블 마스크 생성 시작")

    for uid in tqdm(uids):
        save_path = os.path.join(MASK_SAVE_ROOT, f"{uid}_mask.nii.gz")
        if os.path.exists(save_path): 
            continue

        dicom_dir = os.path.join(DICOM_ROOT_DIR, uid)
        if not os.path.exists(dicom_dir):
            print(f"⚠️ DICOM 폴더 누락: {uid}")
            continue

        try:
            # [Step A] DICOM 로드 및 정렬 (Superior -> Inferior)
            dcm_files = [f for f in os.listdir(dicom_dir) if f.endswith('.png')]
            dcm_data = []
            for f in dcm_files:
                ds = pydicom.dcmread(os.path.join(dicom_dir, f), stop_before_pixels=False)
                dcm_data.append({
                    'pixel': ds.pixel_array.astype(np.float32) * float(getattr(ds, 'RescaleSlope', 1)) + float(getattr(ds, 'RescaleIntercept', 0)),
                    'z_pos': float(ds.ImagePositionPatient[2])
                })
            
            # Z축 기준 정렬 (머리 -> 발)
            dcm_data.sort(key=lambda x: x['z_pos'], reverse=True)
            orig_vol = np.stack([x['pixel'] for x in dcm_data], axis=0)
            orig_z, orig_h, orig_w = orig_vol.shape

            # [Step B] 전처리 (128x224x224)
            vol_win = apply_windowing(orig_vol, WL, WW)
            input_vol = zoom(vol_win, (TARGET_Z/orig_z, TARGET_SIZE/orig_h, TARGET_SIZE/orig_w), order=1)

            # [Step C] 앙상블 추론 (Soft Voting)
            ensemble_probs = np.zeros((TARGET_Z, 8, TARGET_SIZE, TARGET_SIZE), dtype=np.float32)

            for z in range(TARGET_Z):
                img_slice = torch.from_numpy(input_vol[z]).unsqueeze(0)
                z_pos = torch.full((1, TARGET_SIZE, TARGET_SIZE), z / (TARGET_Z - 1))
                input_tensor = torch.cat([img_slice, z_pos], dim=0).unsqueeze(0).to(DEVICE)

                # 5개 모델의 결과를 합산
                combined_prob = torch.zeros((1, 8, TARGET_SIZE, TARGET_SIZE)).to(DEVICE)
                for m in models:
                    output = m(input_tensor)
                    combined_prob += F.softmax(output, dim=1)
                
                ensemble_probs[z] = (combined_prob / len(models)).cpu().numpy()[0]

            # 최종 클래스 결정
            mask_128 = np.argmax(ensemble_probs, axis=1).astype(np.uint8)

            # [Step D] 원본 복원 및 축 교정
            full_mask = zoom(mask_128, (orig_z/TARGET_Z, orig_h/TARGET_SIZE, orig_w/TARGET_SIZE), order=0)
            
            # 축 뒤집기 적용 (Z, Y축)
            full_mask = np.flip(full_mask, axis=(0, 1))

            # [Step E] NIfTI 저장
            mask_img = sitk.GetImageFromArray(full_mask)
            sitk.WriteImage(mask_img, save_path)

        except Exception as e:
            print(f"❌ {uid} 처리 실패: {e}")

if __name__ == '__main__':
    run_ensemble_factory()
    print("\n✨ 타겟 명단 전원에 대한 앙상블 마스크 생성이 완료되었습니다!")

✅ 5-Fold 모델 앙상블 준비 완료

📖 CSV 로드 완료! 타겟 환자 수: 235명
🚀 [old_train/train_images] 폴더에서 앙상블 마스크 생성 시작


 63%|██████▎   | 149/235 [00:00<00:00, 1471.08it/s]

❌ 1.2.826.0.1.3680043.10051 처리 실패: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
❌ 1.2.826.0.1.3680043.10579 처리 실패: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
❌ 1.2.826.0.1.3680043.10678 처리 실패: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
❌ 1.2.826.0.1.3680043.10697 처리 실패: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
❌ 1.2.826.0.1.3680043.10732 처리 실패: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
❌ 1.2.826.0.1.3680043.10815 처리 실패: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force r

100%|██████████| 235/235 [00:00<00:00, 533.03it/s] 

❌ 1.2.826.0.1.3680043.8519 처리 실패: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
❌ 1.2.826.0.1.3680043.8693 처리 실패: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
❌ 1.2.826.0.1.3680043.9447 처리 실패: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
❌ 1.2.826.0.1.3680043.9926 처리 실패: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.
❌ 1.2.826.0.1.3680043.9940 처리 실패: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.

✨ 타겟 명단 전원에 대한 앙상블 마스크 생성이 완료되었습니다!
